In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-dataset-processed/quora_dataset.csv
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/test_tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/requirements.txt
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00004-of-00004.s

In [2]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2024-07-30 00:01:29.023970: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 00:01:29.024082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 00:01:29.159111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from sklearn.model_selection import train_test_split
# Load the train and test data
df = pd.read_csv('/kaggle/input/quora-dataset-processed/quora_dataset.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
test_df=test_df[:250]
train_df=train_df[:4000]

train_df['question'] = train_df['question'].astype(str)
train_df['answer'] = train_df['answer'].astype(str)
test_df['question'] = test_df['question'].astype(str)
test_df['answer'] = test_df['answer'].astype(str)


In [5]:
from datasets import Dataset
from datasets import load_dataset
from transformers import LlamaTokenizer, LlamaForSequenceClassification, Trainer, TrainingArguments

# Convert pandas DataFrames to datasets.Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [6]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on Quora_Qs_Ans', 
    job_type="training"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: feluda0307 (feluda0307-gojo-squad). Use `wandb login --relogin` to force relogin


In [7]:
base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
new_model = "llama-3-8b-fine-tuned-Quora"

In [8]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [9]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

In [11]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [12]:
# Define the function to format chat template
def format_chat_template(row):
    user_content = row["question"] if row["question"] is not None else ""
    assistant_content = row["answer"] if row["answer"] is not None else ""
    
    row_json = [{"role": "user", "content": user_content},
                {"role": "assistant", "content": assistant_content}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Apply the function to train and test datasets
train_dataset = train_dataset.map(format_chat_template, num_proc=4)
test_dataset = test_dataset.map(format_chat_template, num_proc=4)

# Tokenize the datasets
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Print an example
train_dataset['text'][3]
test_dataset['text'][3]


/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

'<|im_start|>user\nwhat deal v us navy abbreviations eg vfa val vaq<|im_end|>\n<|im_start|>assistant\nv french word volare loosely translated means winged aircraft<|im_end|>\n'

In [13]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b2ed54d031b197d982d42bf36e3bc0f0728999af0b2e632356ae059ec5d62399
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [14]:
from datasets import load_metric

rouge = load_metric("rouge",trust_remote_code=True)

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions.argmax(-1)

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    rouge_output = rouge.compute(predictions=pred_str, references=labels_str)
    
    return {
        "rouge1": rouge_output["rouge1"]*100,
        "rouge2": rouge_output["rouge2"]*100,
        "rougeL": rouge_output["rougeL"]*100,
    }

/tmp/ipykernel_34/1212848942.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge",trust_remote_code=True)


In [15]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="no",
#     eval_steps=250,
    logging_steps=100,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
    push_to_hub=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# Function to perform manual garbage collection
def perform_gc():
    gc.collect()
    torch.cuda.empty_cache()

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length=256,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False
)
# # Evaluate the model before training
# results_before_training = trainer.evaluate()
# print("Results before training:", results_before_training)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

In [18]:
# torch.cuda.empty_cache()
# perform_gc()

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
100,4.791500
200,3.893000
300,3.961000
400,3.987400
500,3.929300
600,3.703900
700,3.622700
800,3.632800
900,3.787400
1000,3.822100


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=4000, training_loss=3.5595247497558593, metrics={'train_runtime': 11441.5589, 'train_samples_per_second': 0.35, 'train_steps_per_second': 0.35, 'total_flos': 9.2736011108352e+16, 'train_loss': 3.5595247497558593, 'epoch': 1.0})

In [20]:
# # Evaluate the model after training
# results_after_training = trainer.evaluate()
# print("Results after training:", results_after_training)

In [22]:
wandb.finish()
# perform_gc()
model.config.use_cache = True

In [28]:
import wandb
import json
import os

# Define the path to the run directory
run_dir = "/kaggle/working/wandb/latest-run/files"
summary_path = os.path.join(run_dir, 'wandb-summary.json')

# Load the summary JSON file
with open(summary_path, 'r') as f:
    summary = json.load(f)

# Print the summary
print(summary)


{'train/loss': 3.2427, 'train/grad_norm': 6.079164028167725, 'train/learning_rate': 0.0, 'train/epoch': 1.0, 'train/global_step': 4000, '_timestamp': 1722309326.3270507, '_runtime': 11619.98102068901, '_step': 40, 'train_runtime': 11441.5589, 'train_samples_per_second': 0.35, 'train_steps_per_second': 0.35, 'total_flos': 9.2736011108352e+16, 'train_loss': 3.5595247497558593, '_wandb': {'runtime': 11713}}


In [23]:
new_model_name = "llama-3-8b_fine_tuned"
trainer.model.save_pretrained(new_model_name)
trainer.tokenizer.save_pretrained(new_model_name)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('llama-3-8b_fine_tuned/tokenizer_config.json',
 'llama-3-8b_fine_tuned/special_tokens_map.json',
 'llama-3-8b_fine_tuned/tokenizer.json')

In [ ]:
# torch.cuda.empty_cache()